In [3]:
#######################################################################
#Seattle-Washington-Battle of Neighborhoods - Project - Dependencies
#library for web scraping
# library to handle requests
# library to handle JSON files
#Python library data manipulation and analysis
# convert an address into latitude and longitude values
# tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
# import k-means from clustering stage
# map rendering library
#######################################################################
from bs4 import BeautifulSoup
import requests
import json  
import xml
import pandas as pd 
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print("Successfully imported")

Successfully imported


In [4]:
SeattleWashington_data  = pd.read_csv('SeattleWashington.csv') 
SeattleWashington_data

,Neighborhood,PostalCode
0,Federal Way,98003
1,Bellevue,98005
2,Kirkland,98033
3,Lynnwood,98037
4,Mercer Island,98040
5,Redmond,98052
6,Renton\t,98055
7,Seattle,98101
8,Downtown,98101
9,Capital Hill,98102


In [5]:
################################
# Seattle latitude & longitude
################################
address = 'Seattle, Washington'
geolocator = Nominatim()
location = geolocator.geocode(address)
Seattle_latitude = location.latitude
Seattle_longitude = location.longitude
print('The geograpical coordinate of Seattle home are {}, {}.'.format(Seattle_latitude, Seattle_longitude))

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  """


The geograpical coordinate of Seattle home are 47.6038321, -122.3300624.


In [6]:
####################################
# F2 Parameters
# Foursquare ID, Secret and Version
####################################
Client_ID = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX' 
Client_Secret = 'YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY' 
VERSION = '20190724'

#print('Your credentails:')
#print('CLIENT_ID: ' + Client_ID)
#print('CLIENT_SECRET:' + Client_Secret)

In [7]:
radius = 700 
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    Client_ID, 
    Client_Secret, 
    VERSION, 
    Seattle_latitude, 
   Seattle_longitude, 
    radius, 
   LIMIT)
   
#SeattleWashington_data['Latitude']=latitude #Adding a column in the main dataframe for Latitude  
#SeattleWashington_data['Longitude']=longitude #Adding a column in the main dataframe for Longitude 
#SeattleWashington_data

In [8]:
# To get details from the specfied  Seattle latitude and Longitude
results = requests.get(url).json()
#results

In [9]:
import pandas as pd #Python
SeattleWashington_data_1  = pd.read_csv('SeattleWashington_1.csv') 
SeattleWashington_data_1

,Neighborhood,PostalCode,Latitude,Longitude
0,Federal Way,98003,47.316504,-122.322397
1,Bellevue,98005,47.615044,-122.171758
2,Kirkland,98033,47.668830,-122.192387
3,Lynnwood,98037,47.841952,-122.288181
4,Mercer Island,98040,47.582423,-122.233123
5,Redmond,98052,47.670119,-122.118237
6,Renton,98055,47.462337,-122.205506
7,Seattle,98101,47.608492,-122.336407
8,Downtown,98101,47.608492,-122.336407
9,Capital Hill,98102,47.633822,-122.321545


In [10]:
###########################################################
# Create map of Seattle using latitude and longitude values
###########################################################
map_seattle = folium.Map(location=[Seattle_latitude, Seattle_longitude], zoom_start=10)

# add markers to map
for lat, lng, nei in zip(SeattleWashington_data_1['Latitude'],SeattleWashington_data_1['Longitude'],SeattleWashington_data_1['Neighborhood']):
    label = '{}'.format(nei)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_seattle)  
    
map_seattle

In [11]:
venues=results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)
#nearby_venues.columns

In [12]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [13]:
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
#nearby_venues["venue.categories"][1]

In [14]:
######################################################
#############Getting near by venues###################
######################################################
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues

,name,categories,lat,lng
0,Il Corvo,Italian Restaurant,47.602522,-122.331952
1,Biscuit B*tch,Breakfast Spot,47.603237,-122.332010
2,Columbia Tower Club,Social Club,47.604507,-122.330484
3,Juicy Cafe,Café,47.604329,-122.330958
4,Tat's Delicatessen,Sandwich Place,47.601901,-122.332423
5,Top Pot Doughnuts,Donut Shop,47.604023,-122.332499
6,Hole in the Wall Barbecue,BBQ Joint,47.602525,-122.332009
7,Smith Tower Observation Bar,Cocktail Bar,47.602012,-122.331793
8,Columbia Center Observation Deck (Sky View Obs...,Scenic Lookout,47.604595,-122.330816
9,Red Bowls,Noodle House,47.604654,-122.333211


In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=700):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            Client_ID, 
            Client_Secret, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        venue_results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in venue_results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
SeattleWashington_data

,Neighborhood,PostalCode
0,Federal Way,98003
1,Bellevue,98005
2,Kirkland,98033
3,Lynnwood,98037
4,Mercer Island,98040
5,Redmond,98052
6,Renton\t,98055
7,Seattle,98101
8,Downtown,98101
9,Capital Hill,98102


In [17]:
seattle_venues = getNearbyVenues(names=SeattleWashington_data_1['Neighborhood'],
                                   latitudes=SeattleWashington_data_1['Latitude'],
                                   longitudes=SeattleWashington_data_1['Longitude']
                                  )

Federal Way
Bellevue
Kirkland
Lynnwood
Mercer Island
Redmond
Renton
Seattle
Downtown
Capital Hill
Greenwood
Freemont
Greenlake
International District
Pioneer Square
University District
Laurelhurst
Ballard
South
Queen Anne
Bainbridge Island
Madrona
West Seattle
Alki Beach
Columbia City
Belltown
Northgate
Mount Baker
Magnolia


In [18]:
seattle_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Alki Beach,37,37,37,37,37,37
Bainbridge Island,1,1,1,1,1,1
Ballard,100,100,100,100,100,100
Bellevue,11,11,11,11,11,11
Belltown,100,100,100,100,100,100
Capital Hill,38,38,38,38,38,38
Columbia City,6,6,6,6,6,6
Downtown,100,100,100,100,100,100
Federal Way,40,40,40,40,40,40


In [19]:
# one hot encoding
seattle_onehot = pd.get_dummies(seattle_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
seattle_onehot['Neighborhood'] = seattle_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [seattle_onehot.columns[-1]] + list(seattle_onehot.columns[:-1])
seattle_onehot = seattle_onehot[fixed_columns]
seattle_onehot.head()

,Yoga Studio,ATM,Accessories Store,Advertising Agency,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Assisted Living,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Bus Station,Bus Stop,Café,Camera Store,Candy Store,Caribbean Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Design Studio,Dessert Shop,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food Stand,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Hardware Store,Hawaiian Restaurant,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Home Service,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Indie Theater,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Medical Center,Mediterranean Restaurant,Memorial Site,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Park,Performing Arts Venue,Pet Service,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Post Office,Pub,Public Art,Racetrack,Record Shop,Rental Car Location,Restaurant,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Social Club,Soup Place,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tour Provider,Toy / Game Store,Track,Trail,Tree,Tunnel,Turkish Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Waterfront,Wine Bar,Wine Shop,Winery,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Federal Way,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Federal Way,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [20]:
seattle_grouped = seattle_onehot.groupby('Neighborhood').mean().reset_index()
seattle_grouped.head()

,Neighborhood,Yoga Studio,ATM,Accessories Store,Advertising Agency,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Assisted Living,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Bus Station,Bus Stop,Café,Camera Store,Candy Store,Caribbean Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Design Studio,Dessert Shop,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food Stand,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Hardware Store,Hawaiian Restaurant,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Home Service,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Indie Theater,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Medical Center,Mediterranean Restaurant,Memorial Site,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Park,Performing Arts Venue,Pet Service,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Post Office,Pub,Public Art,Racetrack,Record Shop,Rental Car Location,Restaurant,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Social Club,Soup Place,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tour Provider,Toy / Game Store,Track,Trail,Tree,Tunnel,Turkish Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Waterfront,Wine Bar,Wine Shop,Winery,Women's Store
0,Alki Beach,0.00,0.0,0.0,0.027027,0.0,0.0,0.027027,0.00,0.00,0.027027,0.0,0.0,0.000000,0.000000,0.00,0.000000,0.00,0.0,0.00,0.0,0.027027,0.027027,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.027027,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.108108,0.0,0.0,0.0,0.0,0.027027,0.0,0.00,0.0,0.00,0.000000,0.00,0.00,0.0,0.00,0.00,0.0,0.027027,0.0,0.0,0.0,0.0,0.0,0.0,0.027027,0.0,0.0,0.00,0.0,0.0,0.027027,0.0,0.00,0.0,0.00,0.0,0.027027,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.027027,0.00,0.027027,0.00,0.00,0.00,0.0,0.081081,0.000000,0.0,0.000000,0.0,0.0,0.054054,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.027027,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.027027,0.0,0.108108,0.0,0.0,0.00,0.0,0.027027,0.0,0.00,0.00,0.0,0.0

In [ ]:
num_top_venues = 5
for hood in seattle_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp =seattle_grouped[seattle_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Alki Beach----
                venue  freq
0                Park  0.11
1         Coffee Shop  0.11
2      Ice Cream Shop  0.08
3     Thai Restaurant  0.05
4  Italian Restaurant  0.05


----Bainbridge Island----
          venue  freq
0          Tree   1.0
1   Yoga Studio   0.0
2  Noodle House   0.0
3        Office   0.0
4   Opera House   0.0


----Ballard----
                venue  freq
0         Coffee Shop  0.07
1             Brewery  0.06
2    Sushi Restaurant  0.04
3      Sandwich Place  0.04
4  Mexican Restaurant  0.04


----Bellevue----
                   venue  freq
0       Insurance Office  0.09
1  Vietnamese Restaurant  0.09
2            Coffee Shop  0.09
3             Bagel Shop  0.09
4        Automotive Shop  0.09


----Belltown----
              venue  freq
0               Bar  0.07
1       Coffee Shop  0.05
2  Sushi Restaurant  0.05
3       Pizza Place  0.04
4  Sculpture Garden  0.03


----Capital Hill----
                venue  freq
0              Garden  0.08
1       

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = seattle_grouped['Neighborhood']

for ind in np.arange(seattle_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(seattle_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alki Beach,Park,Coffee Shop,Ice Cream Shop,Italian Restaurant,Thai Restaurant,Pizza Place,Beach,Beach Bar,Brewery,Seafood Restaurant
1,Bainbridge Island,Tree,Ethiopian Restaurant,Forest,Food Truck,Food Stand,Flower Shop,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant
2,Ballard,Coffee Shop,Brewery,New American Restaurant,Sandwich Place,Sushi Restaurant,Mexican Restaurant,Ice Cream Shop,Bar,Pizza Place,Cocktail Bar
3,Bellevue,Insurance Office,Auto Garage,Mexican Restaurant,Other Repair Shop,Deli / Bodega,Coffee Shop,Spa,Bagel Shop,Automotive Shop,Indian Restaurant
4,Belltown,Bar,Sushi Restaurant,Coffee Shop,Pizza Place,Vietnamese Restaurant,Sculpture Garden,Movie Theater,Breakfast Spot,Gym,Italian Restaurant


In [ ]:
##############################################################
# set number of clusters
# Using k-means to cluster the neighborhood into 4 clusters.
##############################################################
kclusters = 4

seattle_grouped_clustering = seattle_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(seattle_grouped_clustering)

# check cluster labels generated for each row in the dataframe
#kmeans.labels_

In [ ]:
#############################################################################
# Merge Seattle with Seattle to add latitude/longitude for each neighborhood
#############################################################################
seattle_merged = SeattleWashington_data_1
seattle_merged['Cluster Labels'] = kmeans.labels_
seattle_merged = seattle_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
seattle_merged.head()# check the last columns!

,Neighborhood,PostalCode,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Federal Way,98003,47.316504,-122.322397,0,Coffee Shop,Mexican Restaurant,Pet Store,Optical Shop,Miscellaneous Shop,Sports Bar,Fast Food Restaurant,Gas Station,Fried Chicken Joint,Pizza Place
1,Bellevue,98005,47.615044,-122.171758,2,Insurance Office,Auto Garage,Mexican Restaurant,Other Repair Shop,Deli / Bodega,Coffee Shop,Spa,Bagel Shop,Automotive Shop,Indian Restaurant
2,Kirkland,98033,47.668830,-122.192387,0,Café,Music Venue,Asian Restaurant,Sandwich Place,Hotel,Grocery Store,Brewery,Plaza,Forest,Frozen Yogurt Shop
3,Lynnwood,98037,47.841952,-122.288181,0,Sandwich Place,Coffee Shop,Sporting Goods Shop,Café,Bank,Fast Food Restaurant,BBQ Joint,Automotive Shop,Smoke Shop,Park
4,Mercer Island,98040,47.582423,-122.233123,0,Coffee Shop,Pharmacy,Pizza Place,Gas Station,Sandwich Place,Thai Restaurant,Yoga Studio,Mexican Restaurant,Sushi Restaurant,Supermarket


In [ ]:
##############################################
# Create map Seattle using latitude,longitude
##############################################

map_clusters = folium.Map(location=[Seattle_latitude,Seattle_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
colors_array = cm.rainbow(np.linspace(0, 1, kclusters))
rainbow = [colors.rgb2hex(i) for i in colors_array]
print(rainbow)
# add markers to the map
markers_colors = []
for lat, lon, nei , cluster in zip(seattle_merged['Latitude'], seattle_merged['Longitude'], seattle_merged['Neighborhood'], seattle_merged['Cluster Labels']):
    label = folium.Popup(str(nei) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

['#8000ff', '#2adddd', '#d4dd80', '#ff0000']


In [ ]:
df1=seattle_merged.loc[seattle_merged['Cluster Labels'] == 0,seattle_merged.columns[[0] + list(range(5, seattle_merged.shape[1]))]]
df1.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Federal Way,Coffee Shop,Mexican Restaurant,Pet Store,Optical Shop,Miscellaneous Shop,Sports Bar,Fast Food Restaurant,Gas Station,Fried Chicken Joint,Pizza Place
2,Kirkland,Café,Music Venue,Asian Restaurant,Sandwich Place,Hotel,Grocery Store,Brewery,Plaza,Forest,Frozen Yogurt Shop
3,Lynnwood,Sandwich Place,Coffee Shop,Sporting Goods Shop,Café,Bank,Fast Food Restaurant,BBQ Joint,Automotive Shop,Smoke Shop,Park
4,Mercer Island,Coffee Shop,Pharmacy,Pizza Place,Gas Station,Sandwich Place,Thai Restaurant,Yoga Studio,Mexican Restaurant,Sushi Restaurant,Supermarket
5,Redmond,Bakery,Mexican Restaurant,Furniture / Home Store,Burger Joint,Coffee Shop,Gym / Fitness Center,Sandwich Place,Ice Cream Shop,Cosmetics Shop,Accessories Store


In [ ]:
df2=seattle_merged.loc[seattle_merged['Cluster Labels'] == 1,seattle_merged.columns[[0] + list(range(5, seattle_merged.shape[1]))]]
df2

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Pioneer Square,Coffee Shop,Café,Hotel,Cocktail Bar,Seafood Restaurant,Donut Shop,Italian Restaurant,Scenic Lookout,Pizza Place,Salad Place
27,Mount Baker,Park,Video Store,Fast Food Restaurant,Bus Station,Dog Run,Mexican Restaurant,Playground,Ethiopian Restaurant,Sandwich Place,Garden


In [ ]:
df3=seattle_merged.loc[seattle_merged['Cluster Labels'] == 2,seattle_merged.columns[[0] + list(range(5, seattle_merged.shape[1]))]]
df3

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Bellevue,Insurance Office,Auto Garage,Mexican Restaurant,Other Repair Shop,Deli / Bodega,Coffee Shop,Spa,Bagel Shop,Automotive Shop,Indian Restaurant
16,Laurelhurst,Beach,Women's Store,Event Space,Forest,Food Truck,Food Stand,Flower Shop,Fish Market,Fish & Chips Shop,Filipino Restaurant


In [ ]:
df4=seattle_merged.loc[seattle_merged['Cluster Labels'] == 3,seattle_merged.columns[[0] + list(range(5, seattle_merged.shape[1]))]]
df4

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Renton,Coffee Shop,Insurance Office,Asian Restaurant,Park,Bar,Pizza Place,Grocery Store,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant


In [ ]:
new_seattle=seattle_merged.set_index("Neighborhood",drop=True)
new_seattle.head()

,PostalCode,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Neighborhood,,,,,,,,,,,,,,
Federal Way,98003,47.316504,-122.322397,0,Coffee Shop,Mexican Restaurant,Pet Store,Optical Shop,Miscellaneous Shop,Sports Bar,Fast Food Restaurant,Gas Station,Fried Chicken Joint,Pizza Place
Bellevue,98005,47.615044,-122.171758,2,Insurance Office,Auto Garage,Mexican Restaurant,Other Repair Shop,Deli / Bodega,Coffee Shop,Spa,Bagel Shop,Automotive Shop,Indian Restaurant
Kirkland,98033,47.668830,-122.192387,0,Café,Music Venue,Asian Restaurant,Sandwich Place,Hotel,Grocery Store,Brewery,Plaza,Forest,Frozen Yogurt Shop
Lynnwood,98037,47.841952,-122.288181,0,Sandwich Place,Coffee Shop,Sporting Goods Shop,Café,Bank,Fast Food Restaurant,BBQ Joint,Automotive Shop,Smoke Shop,Park
Mercer Island,98040,47.582423,-122.233123,0,Coffee Shop,Pharmacy,Pizza Place,Gas Station,Sandwich Place,Thai Restaurant,Yoga Studio,Mexican Restaurant,Sushi Restaurant,Supermarket


In [ ]:
new_seattle_population=pd.DataFrame({
   
    "White":[62408,12065,29436,18512,18494,41028,11769,7734,7734,17951,40141,40141,40141,6638,6638,33597,33597,19093,17618,17618,21763,21763,20253,20253,14855,9447,26585,13040,17705],
"Black":[7001, 547, 696,1642,407, 1360, 3552,888,888, 754,1592,1592,1592,2633, 2633,1353,1353,655,1051,1051,236,236,666,666, 12333,1039,3531,5594,474],
"Hispanic":[6514,1213,1667,2544,634, 4381,2089,612,612,1212,2318, 2318,2318, 949,949, 2078, 2078,1043,1065, 1065,887,887,989,989,3440,808,3010,3055,869],
 "Asian": [10836,5012, 4484,6040,4279,15539,6140,1465,1465, 2176,4775,4775,4775,3404, 3404,9902,9902,1507,2210,2210,1263,1263,1575,1575,15057,2098,6656,7429,1748],
  "Hawaiian":[875,74,180,333,84,222,369,70,70, 95,206,206,206,93,93,324, 324,106,124,124,118,118,115,115,584,96,373, 210,125],  
"Indian":[783,157,366,599,169,618,397,285,285,339,783,783,783,453,453,636,636,440,317,317,417,417,408,408,924,309,912,657,288],

}
)
new_seattle_population.head()

,White,Black,Hispanic,Asian,Hawaiian,Indian
0,62408,7001,6514,10836,875,783
1,12065,547,1213,5012,74,157
2,29436,696,1667,4484,180,366
3,18512,1642,2544,6040,333,599
4,18494,407,634,4279,84,169


In [ ]:
new_seattle_population["Neighborhood"]=seattle_merged["Neighborhood"]

In [ ]:
import matplotlib.pyplot as plt
new_seattle_population=new_seattle_population[['Neighborhood','White','Black','Asian','Hispanic','Indian','Hawaiian']]
new_seattle_population.set_index('Neighborhood',inplace=True,drop=True)

new_seattle_population.plot(kind='bar',figsize=(25,7))
plt.show()

<Figure size 2500x700 with 1 Axes>

In [ ]:
#Source:https://www.greatschools.org
Seattle_school_ratings=pd.DataFrame({"Neighborhood":SeattleWashington_data_1["Neighborhood"],
                                      "Top School Rating":[8,9,9,8,10,10,7,10,0,0,0,0,7,0,0,0,6,0,0,6,10,5,5,9,4,0,0,3,0]})

In [ ]:
#Seattle_school_ratings.sort_values(by='Top School Rating',ascending=False)
Seattle_school_ratings.set_index('Neighborhood',inplace=True,drop=True)

In [ ]:
print("Neighborhood - School Rating")
Seattle_school_ratings.plot(kind='bar',figsize=(10,5),color='lightBlue',alpha=0.75);

Neighborhood - School Rating


In [ ]:
import pandas as pd
Seattle_Avg_HousingPrice=pd.DataFrame({"Neighborhood":SeattleWashington_data_1["Neighborhood"],
                                      "Average_Housing_Price":[172050.0,343500.0,297900.0,202000.0,
                                                               573900.0,279400.0,178000.0,275000.0,275000.0,477000.0,279200.0,279200.0,
                                                               279200.0,225000.0,225000.0,370500.0,370500.0,255400.0,433500.0,433500.0,
                                                               335000.0,335000.0,286600.0,286600.0,187300.0,175000.0,225900.0,219400.0,363400.0]})

In [ ]:
Seattle_Avg_HousingPrice.set_index('Neighborhood',inplace=True,drop=True)

In [ ]:
print("House Pricing")
Seattle_Avg_HousingPrice.plot(kind='bar',figsize=(10,7),color='ORANGE',alpha=0.75);